# Summary

Test out new batch query functionality.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import openai
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    GPTBackend, query_kwargs_grid, MOCKS, postprocess_gpt_response, \
    containerize, truncate_at_first_stop, query_gpt_mock
from jabberwocky.utils import strip
from htools import *

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


In [5]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [6]:
gpt = GPTBackend()
gpt.switch('repeat')

Switching openai backend to "repeat".


In [7]:
def postprocess_response(response, n, trunc_full=True, strip_output=True, 
                         **kwargs):
    text, full_response = containerize(*response)
    print('TEXT:', text)
    print('FULL:', full_response)

    # Manually check for stop phrases because most backends either don't
    # or truncate AFTER the stop phrase which is rarely what we want.
    stop = kwargs.get('stop', [])
    clean_text = []
    clean_full = []
    for i, (text_, resp_) in enumerate(zip(text, full_response)):
        text_ = truncate_at_first_stop(
            text_,
            stop_phrases=stop,
            finish_reason=resp_.get('finish_reason', ''),
            trunc_full=trunc_full,
            trunc_partial=True
        )
        clean_text.append(strip(text_, strip_output))
        clean_full.append({**resp_, 'prompt_index': i // n})

    return clean_text, clean_full

In [37]:
def test_backend(backend, results=None):
    def gen():
        yield from results.kwargs
            
    full_keys = ('prompt_index', 'index', 'finish_reason')
    with gpt(backend):
        kwargs_list = []
        resp_list = []
        gen_kwargs = gen if results else query_kwargs_grid
        for i, kwargs in enumerate(gen_kwargs(), start=1):
            print(f'\n\n{i}.')
            if isinstance(kwargs['prompt'], str):
                np = 1
            else:
                np = len(kwargs['prompt'])
            expected_prompt_idx = list(range(np))
                
            # Get completion, either from new query or cached result.
            if results:
                res = results.res[i - 1]
            else:
                res = gpt.query(**kwargs)
                
            # Print results.
            if kwargs['stream']:
                cur = []
                actual_prompt_idx = []
                for tok, tok_full in res:
                    cur.append((tok, tok_full))
                    print(tok)
                    print('\t' + str(select(tok_full, keep=full_keys)) + '\n')
                    if tok_full['finish_reason']: print('\n---\n')
                    actual_prompt_idx.append(tok_full.get('prompt_index', -1))
            else:
                texts, fulls = res
                print('TEXTS:', texts)
                print('FULLS:',
                      [{key: full.get(key) for key in full_keys}
                       for full in fulls])
                actual_prompt_idx = [full['prompt_index'] for full in fulls]
                cur = res
            
            actual_prompt_idx = sorted(set(actual_prompt_idx))
            assert actual_prompt_idx == expected_prompt_idx, \
                    f'Expected prompt indices {expected_prompt_idx}, ' \
                    f'got {actual_prompt_idx}'
            print(spacer())

            kwargs_list.append(kwargs)
            resp_list.append(cur)
    return Results(kwargs=kwargs_list, res=resp_list)

Issues

[X] - no prompt_index in stream=True mode for either repeat or banana (pretty sure not for paid backends either)

In [38]:
repeat_res = test_backend('repeat')

Switching openai backend to "repeat".
np>1: False
nc>1: False
stream: False


1.
{'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
TEXTS: ['YESTERDAY WAS']
FULLS: [{'prompt_index': 0, 'index': None, 'finish_reason': None}]

-------------------------------------------------------------------------------

np>1: False
nc>1: False
stream: True


2.
{'n': 1, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'repeat', 'query_func': 'query_gpt_repeat'}}
YESTERDAY 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

WAS 
	{'prompt_index': 0, 'index': 0, 'finish_reason': 'dummy'}


---


-------------------------------------------------------------------------------

np>1: False
nc>1: True
stream: False


3.
{'n': 2, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', '

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:415: UserWarning: Unused kwargs {'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3} received by query_gpt_repeat.
  warnings.warn(f'Unused kwargs {kwargs} received by query_gpt_repeat.')
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:415: UserWarning: Unused kwargs {'n': 1, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3} received by query_gpt_repeat.
  warnings.warn(f'Unused kwargs {kwargs} received by query_gpt_repeat.')
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:415: UserWarning: Unused kwargs {'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3} received by query_gpt_repeat.
  warnings.warn(f'Unused kwargs {kwargs} received by query_gpt_repeat.')
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:415: UserWarning: Unused kwargs {'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3} received by query_gpt_repeat.
  warnings.warn(f'Unused kw

In [39]:
# Even though it's free, try to avoid hitting the API too much just in case 
# they have some rate limit.
banana_res = test_backend('banana')
save(banana_res, 'data/tmp/banana_res.pkl')
banana_res = load('data/tmp/banana_res.pkl')

Switching openai backend to "banana".
np>1: False
nc>1: False
stream: False


1.
{'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'banana', 'query_func': 'query_gpt_banana'}}


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:470: UserWarning: query_gpt_banana received unused kwargs {'n': 1, 'stream': False, 'engine_i': 0, 'logprobs': 3}.
  warnings.warn(f'query_gpt_banana received unused kwargs {kwargs}.')


TEXTS: ['the one year']
FULLS: [{'prompt_index': 0, 'index': None, 'finish_reason': None}]

-------------------------------------------------------------------------------

np>1: False
nc>1: False
stream: True


2.
{'n': 1, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'banana', 'query_func': 'query_gpt_banana'}}


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:470: UserWarning: query_gpt_banana received unused kwargs {'n': 1, 'stream': True, 'engine_i': 0, 'logprobs': 3}.
  warnings.warn(f'query_gpt_banana received unused kwargs {kwargs}.')


 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

the 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

first 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

day 
	{'prompt_index': 0, 'index': 0, 'finish_reason': 'dummy'}


---


-------------------------------------------------------------------------------

np>1: False
nc>1: True
stream: False


3.
{'n': 2, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'banana', 'query_func': 'query_gpt_banana'}}


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:470: UserWarning: query_gpt_banana received unused kwargs {'stream': False, 'engine_i': 0, 'logprobs': 3}.
  warnings.warn(f'query_gpt_banana received unused kwargs {kwargs}.')


TEXTS: ['a big day', 'my last day']
FULLS: [{'prompt_index': 0, 'index': None, 'finish_reason': None}, {'prompt_index': 0, 'index': None, 'finish_reason': None}]

-------------------------------------------------------------------------------

np>1: False
nc>1: True
stream: True


4.
{'n': 2, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'banana', 'query_func': 'query_gpt_banana'}}


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:470: UserWarning: query_gpt_banana received unused kwargs {'stream': True, 'engine_i': 0, 'logprobs': 3}.
  warnings.warn(f'query_gpt_banana received unused kwargs {kwargs}.')


 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

my 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

birthday. 
	{'prompt_index': 0, 'index': 0, 'finish_reason': 'dummy'}


---

 
	{'prompt_index': 0, 'index': 1, 'finish_reason': None}

my 
	{'prompt_index': 0, 'index': 1, 'finish_reason': None}

first 
	{'prompt_index': 0, 'index': 1, 'finish_reason': None}

day 
	{'prompt_index': 0, 'index': 1, 'finish_reason': 'dummy'}


---


-------------------------------------------------------------------------------

np>1: True
nc>1: False
stream: False


5.
{'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'banana', 'query_func': 'query_gpt_banana'}}
{'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'How many', 'meta': {'backend_name': 'banana', 'query_func': 'query_gpt_banana'}}
TEXTS: ['the first day', 'minutes are there']
FULLS: [{'prompt_index': 0, 'index': None, 

In [41]:
banana_res = test_backend('banana', results=banana_res)

Switching openai backend to "banana".


1.
TEXTS: ['the one year']
FULLS: [{'prompt_index': 0, 'index': None, 'finish_reason': None}]

-------------------------------------------------------------------------------



2.
 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

the 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

first 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

day 
	{'prompt_index': 0, 'index': 0, 'finish_reason': 'dummy'}


---


-------------------------------------------------------------------------------



3.
TEXTS: ['a big day', 'my last day']
FULLS: [{'prompt_index': 0, 'index': None, 'finish_reason': None}, {'prompt_index': 0, 'index': None, 'finish_reason': None}]

-------------------------------------------------------------------------------



4.
 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

my 
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

birthday. 
	{'prompt_index': 0, 'index': 0, 'finish_reason': 'dummy

In [42]:
mock_res = test_backend('mock')

Switching openai backend to "mock".
np>1: False
nc>1: False
stream: False


1.
{'n': 1, 'stream': False, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'mock', 'query_func': 'query_gpt_mock'}}
TEXTS: ['a typical rainy']
FULLS: [{'prompt_index': 0, 'index': 0, 'finish_reason': 'length'}]

-------------------------------------------------------------------------------

np>1: False
nc>1: False
stream: True


2.
{'n': 1, 'stream': True, 'engine_i': 0, 'max_tokens': 3, 'logprobs': 3, 'prompt': 'Yesterday was', 'meta': {'backend_name': 'mock', 'query_func': 'query_gpt_mock'}}
 a
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

 bit
	{'prompt_index': 0, 'index': 0, 'finish_reason': None}

 busy
	{'prompt_index': 0, 'index': 0, 'finish_reason': 'length'}


---


-------------------------------------------------------------------------------

np>1: False
nc>1: True
stream: False


3.
{'n': 2, 'stream': False, 'engine_i': 0, 'max_tokens

In [10]:
# hf_res = test_backend('huggingface')

## Scratch

In [51]:
# 2 prompts, 1 completion per prompt, stream=False
p2c1s0 = MOCKS[True, False, False]
p1c2s0 = MOCKS[False, True, False]

In [52]:
texts, fulls = containerize(*postprocess_gpt_response(p2c1s0))
postprocess_response((texts, fulls), n=1)

(['24 years ago', 'months did it'],
 [{'finish_reason': 'length',
   'index': 0,
   'logprobs': <OpenAIObject at 0x125bacaf0> JSON: {
     "text_offset": [
       0,
       3,
       9
     ],
     "token_logprobs": [
       -8.1484375,
       -1.931640625,
       -0.8271484375
     ],
     "tokens": [
       " 24",
       " years",
       " ago"
     ],
     "top_logprobs": [
       {
         " a": -1.3818359375,
         " my": -2.384765625,
         " the": -1.8720703125
       },
       {
         " hours": -1.21484375,
         " years": -1.931640625,
         "th": -1.8212890625
       },
       {
         " ago": -0.8271484375,
         " since": -1.0576171875,
         " to": -2.794921875
       }
     ]
   },
   'text': ' 24 years ago',
   'token_index': 0,
   'prompt_index': 0},
  {'finish_reason': 'length',
   'index': 1,
   'logprobs': <OpenAIObject at 0x125bacca8> JSON: {
     "text_offset": [
       0,
       7,
       11
     ],
     "token_logprobs": [
       -5.464843

In [73]:
texts, fulls = postprocess_gpt_response(p1c2s0)
postprocess_response((texts, fulls), n=2)

TEXT: [' my mother�', ' the Academy Awards']
FULL: [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x1245f1990> JSON: {
  "text_offset": [
    0,
    3,
    10
  ],
  "token_logprobs": [
    -2.384765625,
    -5.4375,
    -0.44677734375
  ],
  "tokens": [
    " my",
    " mother",
    "\ufffd"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    },
    {
      " birthday": -2.107421875,
      " first": -1.3818359375,
      " last": -2.130859375
    },
    {
      "'s": -1.509765625,
      "-": -2.630859375,
      "\ufffd": -0.44677734375
    }
  ]
}, 'text': ' my mother�', 'token_index': 0}, {'finish_reason': 'length', 'index': 1, 'logprobs': <OpenAIObject at 0x1245f1b48> JSON: {
  "text_offset": [
    0,
    4,
    12
  ],
  "token_logprobs": [
    -1.8720703125,
    -8.796875,
    -0.1270751953125
  ],
  "tokens": [
    " the",
    " Academy",
    " Awards"
  ],
  "top_logprobs": [
    {
      " a":

(['my mother�', 'the Academy Awards'],
 [{'finish_reason': 'length',
   'index': 0,
   'logprobs': <OpenAIObject at 0x1245f1990> JSON: {
     "text_offset": [
       0,
       3,
       10
     ],
     "token_logprobs": [
       -2.384765625,
       -5.4375,
       -0.44677734375
     ],
     "tokens": [
       " my",
       " mother",
       "\ufffd"
     ],
     "top_logprobs": [
       {
         " a": -1.3818359375,
         " my": -2.384765625,
         " the": -1.8720703125
       },
       {
         " birthday": -2.107421875,
         " first": -1.3818359375,
         " last": -2.130859375
       },
       {
         "'s": -1.509765625,
         "-": -2.630859375,
         "\ufffd": -0.44677734375
       }
     ]
   },
   'text': ' my mother�',
   'token_index': 0,
   'prompt_index': 0},
  {'finish_reason': 'length',
   'index': 1,
   'logprobs': <OpenAIObject at 0x1245f1b48> JSON: {
     "text_offset": [
       0,
       4,
       12
     ],
     "token_logprobs": [
       -1.

In [74]:
texts

[' my mother�', ' the Academy Awards']

In [115]:
with gpt('mock'):
    res = gpt.query('ac', n=1, stream=False)

Switching openai backend to "mock".
{'n': 1, 'stream': False, 'prompt': 'ac', 'meta': {'backend_name': 'mock', 'query_func': 'query_gpt_mock'}}
>>> response: ([' a typical rainy'], [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x125a06e08> JSON: {
  "text_offset": [
    0,
    2,
    10
  ],
  "token_logprobs": [
    -1.3818359375,
    -6.06640625,
    -5.671875
  ],
  "tokens": [
    " a",
    " typical",
    " rainy"
  ],
  "top_logprobs": [
    {
      " a": -1.3818359375,
      " my": -2.384765625,
      " the": -1.8720703125
    },
    {
      " big": -1.83984375,
      " day": -2.53125,
      " very": -2.916015625
    },
    {
      " Monday": -2.974609375,
      " day": -1.095703125,
      " work": -2.94140625
    }
  ]
}, 'text': ' a typical rainy', 'token_index': 0}])
TEXT: [' a typical rainy']
FULL: [{'finish_reason': 'length', 'index': 0, 'logprobs': <OpenAIObject at 0x125a06e08> JSON: {
  "text_offset": [
    0,
    2,
    10
  ],
  "token_logprobs":

In [116]:
res

(['a typical rainy'],
 [{'finish_reason': 'length',
   'index': 0,
   'logprobs': <OpenAIObject at 0x125a06e08> JSON: {
     "text_offset": [
       0,
       2,
       10
     ],
     "token_logprobs": [
       -1.3818359375,
       -6.06640625,
       -5.671875
     ],
     "tokens": [
       " a",
       " typical",
       " rainy"
     ],
     "top_logprobs": [
       {
         " a": -1.3818359375,
         " my": -2.384765625,
         " the": -1.8720703125
       },
       {
         " big": -1.83984375,
         " day": -2.53125,
         " very": -2.916015625
       },
       {
         " Monday": -2.974609375,
         " day": -1.095703125,
         " work": -2.94140625
       }
     ]
   },
   'text': ' a typical rainy',
   'token_index': 0,
   'prompt_index': 0}])